# FileStream, countByWindow, reduceByKeyAndWindow, checkpoing using  old (DStream) API

In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.streaming import * 
from pyspark import SparkConf, SparkContext

In [3]:
#sc = SparkContext(appName="WindowOperations", master="local[4]")
sc = SparkContext.getOrCreate()

In [4]:
ssc = StreamingContext(sc, 5)

In [5]:
lines = ssc.textFileStream("data")

In [6]:
ssc.checkpoint("checkpoint")

In [7]:
words = lines.flatMap(lambda x: x.split(" "))

In [8]:
mapped_words = words.map(lambda x: (x,1))

## 1. Windows Operation

#### Here slideDuration should be 2 times bigger than batchDuration

In [9]:
window = mapped_words.window(windowDuration=30, slideDuration=10)

In [10]:
window.pprint(10)

In [11]:
ssc.start()

-------------------------------------------
Time: 2019-04-27 12:17:40
-------------------------------------------

-------------------------------------------
Time: 2019-04-27 12:17:50
-------------------------------------------
('Once', 1)
('upon', 1)
('a', 1)
('time', 1)
('there', 1)
('dwelt', 1)
('on', 1)
('the', 1)
('outskirts', 1)
('of', 1)
...

-------------------------------------------
Time: 2019-04-27 12:18:00
-------------------------------------------
('Once', 1)
('upon', 1)
('a', 1)
('time', 1)
('there', 1)
('dwelt', 1)
('on', 1)
('the', 1)
('outskirts', 1)
('of', 1)
...

-------------------------------------------
Time: 2019-04-27 12:18:10
-------------------------------------------
('Once', 1)
('upon', 1)
('a', 1)
('time', 1)
('there', 1)
('dwelt', 1)
('on', 1)
('the', 1)
('outskirts', 1)
('of', 1)
...

-------------------------------------------
Time: 2019-04-27 12:18:20
-------------------------------------------

-------------------------------------------
Time: 2019-0

In [12]:
ssc.stop(stopSparkContext= True, stopGraceFully= True)

-------------------------------------------
Time: 2019-04-27 12:18:40
-------------------------------------------



## 2. countByWindow

CountByWindow returns a sliding window count of elements in the stream. We have 10 elements in the first window and then comes 15 elements in the second window, in total it calculates the 25 elements.

In [20]:
countByWindow = mapped_words.countByWindow(windowDuration=30, slideDuration=10)

In [21]:
countByWindow.pprint(10)

In [22]:
ssc.start()

-------------------------------------------
Time: 2019-04-27 12:19:10
-------------------------------------------

-------------------------------------------
Time: 2019-04-27 12:19:20
-------------------------------------------
1890

-------------------------------------------
Time: 2019-04-27 12:19:30
-------------------------------------------
3780

-------------------------------------------
Time: 2019-04-27 12:19:40
-------------------------------------------
3780

-------------------------------------------
Time: 2019-04-27 12:19:50
-------------------------------------------
3780

-------------------------------------------
Time: 2019-04-27 12:20:00
-------------------------------------------
5670



In [23]:
ssc.stop(stopSparkContext= True, stopGraceFully= True)

-------------------------------------------
Time: 2019-04-27 12:20:10
-------------------------------------------
5670



Here we add HanselStory.txt file and the program reads 1890 words. After that, we copy text file and it shows 3780 which is the last added txt file (3 files) counts.

## 3. reduceByKeyAndWindow

In [20]:
reduceByKeyAndWindow = mapped_words.reduceByKeyAndWindow(
    lambda x,y: (int(x) + int(y)), \
    lambda x,y: (int(x)- int(y)), \
    windowDuration=30, slideDuration=10) \
.map(lambda x: (x[1], x[0])) \
.transform(lambda x: x.sortByKey(False))

In [21]:
reduceByKeyAndWindow.pprint(10)

In [22]:
ssc.start()

-------------------------------------------
Time: 2019-04-27 12:27:35
-------------------------------------------

-------------------------------------------
Time: 2019-04-27 12:27:45
-------------------------------------------
(113, 'the')
(91, 'and')
(44, 'to')
(42, 'a')
(34, 'they')
(31, 'of')
(27, 'had')
(19, 'was')
(19, 'in')
(19, 'he')
...

-------------------------------------------
Time: 2019-04-27 12:27:55
-------------------------------------------
(226, 'the')
(182, 'and')
(88, 'to')
(84, 'a')
(68, 'they')
(62, 'of')
(54, 'had')
(38, 'was')
(38, 'in')
(38, 'he')
...

-------------------------------------------
Time: 2019-04-27 12:28:05
-------------------------------------------
(339, 'the')
(273, 'and')
(132, 'to')
(126, 'a')
(102, 'they')
(93, 'of')
(81, 'had')
(57, 'was')
(57, 'in')
(57, 'he')
...



In [23]:
ssc.stop(stopSparkContext= True, stopGraceFully= True)

-------------------------------------------
Time: 2019-04-27 12:28:15
-------------------------------------------
(226, 'the')
(182, 'and')
(88, 'to')
(84, 'a')
(68, 'they')
(62, 'of')
(54, 'had')
(38, 'was')
(38, 'in')
(38, 'he')
...



Here words are reduced (counted) using reduceByKeyAndWindows() function. Then we copy the text file every 10 seconds. Our calculating slide is every 10 seconds so that in stop action we get only the last two file results.